In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Function to split the dataset
def split_data(df, train_ratio=0.8):
    df.dropna(inplace=True)  # Remove rows with NaN values

    # Scale features
    feature_scaler = StandardScaler()
    scaled_features = feature_scaler.fit_transform(df.drop(columns=['Close']))
    scaled_data = pd.DataFrame(scaled_features, columns=df.drop(columns=['Close']).columns, index=df.index)
    scaled_data['Close'] = df['Close']  # Keep original target for splitting

    # Scale the target variable
    target_scaler = StandardScaler()
    scaled_data['Close'] = target_scaler.fit_transform(df['Close'].values.reshape(-1, 1)).ravel()

    # Split data
    split_idx = int(len(df) * train_ratio)
    train_data = scaled_data.iloc[:split_idx]
    test_data = scaled_data.iloc[split_idx:]

    # Separate features and targets
    X_train = train_data.drop(columns=['Close'])
    y_train = train_data['Close']
    X_test = test_data.drop(columns=['Close'])
    y_test = test_data['Close']

    return X_train, y_train, X_test, y_test, test_data.index, feature_scaler, target_scaler

# Function to train the XGBoost model
def train_xgboost(X_train, y_train):
    model = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        objective='reg:squarederror',
        random_state=42
    )
    model.fit(X_train, y_train)
    return model

# Function to calculate log-return volatility
def calculate_log_volatility(y_values):
    log_returns = np.log(y_values[1:] / y_values[:-1])  # Logarithmic returns
    return np.std(log_returns)  # Volatility as standard deviation of log returns

def evaluate_multiple_start_points(model, X_test, y_test, test_index, n_days_to_predict, num_start_points, feature_scaler, target_scaler):
    errors = []
    volatilities = []

    # Select start indices (you can adjust the range as needed)
    start_indices = range(0, len(X_test) - n_days_to_predict, 5)  # Make sure we have enough data for predictions

    for start_idx in start_indices:
        end_idx = start_idx + n_days_to_predict

        # Select the future data for prediction (not including past data)
        X_sub = X_test.iloc[start_idx:end_idx][['Date']]  # Select only the Date column
        y_true = y_test.iloc[start_idx:end_idx]

        # Predict and invert scaling for the target
        predictions = model.predict(X_sub)
        predictions = target_scaler.inverse_transform(predictions.reshape(-1, 1)).ravel()
        y_true_unscaled = target_scaler.inverse_transform(y_true.values.reshape(-1, 1)).ravel()

        # Calculate MSE
        mse = mean_squared_error(y_true_unscaled, predictions)
        errors.append(mse)

        # Calculate log-return volatility
        log_volatility = calculate_log_volatility(y_true_unscaled)
        volatilities.append(log_volatility)

        # Plot individual predictions
        plt.figure(figsize=(10, 6))
        # Plot only future data
        plt.plot(test_index[start_idx:end_idx], y_true_unscaled, label="True Data", color="green")
        plt.plot(test_index[start_idx:end_idx], predictions, label="Predictions", color="orange")
        plt.xlabel("Date")
        plt.ylabel("Close Price")
        plt.title(f"Predictions from Index {start_idx} to {end_idx}")
        plt.legend()
        os.makedirs("TSLA_pred/xdboost/nolag", exist_ok=True)
        plt.savefig(f"TSLA_pred/xdboost/nolag/start_{start_idx}.png")
        plt.close()

    # Calculate average MSE
    avg_mse = np.mean(errors)

    # Scatter plot: log-return volatility vs MSE
    plt.figure(figsize=(10, 6))
    plt.scatter(volatilities, errors, marker="o", color="blue")
    for i, idx in enumerate(start_indices):
        plt.text(volatilities[i], errors[i], f"Start {idx}", fontsize=9)
    plt.xlabel("Log-Return Volatility (std)")
    plt.ylabel("Mean Squared Error (MSE)")
    plt.title("Volatility vs MSE for Multiple Starting Points")
    plt.grid(True)

    # Add legend with average MSE
    plt.legend([f"Average MSE: {avg_mse:.4f}"], loc="lower right")
    plt.tight_layout()

    # Save the plot
    os.makedirs("TSLA_pred/xdboost/nolag", exist_ok=True)
    plt.savefig("TSLA_pred/xdboost/nolag/log_volatility_vs_mse.png")
    plt.show()


# Main function
def main():
    n_days_to_predict = 15  # Number of days to predict for each starting point
    num_start_points = 15  # Number of unique starting points
    path = "TSLA.csv"  # Path to your stock data file

    # Load and preprocess data
    headers = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
    df = pd.read_csv(path, names=headers, skiprows=1)
    df.replace("null", np.nan, inplace=True)
    df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]] = df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].astype(float)
    df.set_index("Date", inplace=True)

    # Split the dataset into training and testing sets
    X_train, y_train, X_test, y_test, test_index, feature_scaler, target_scaler = split_data(df)
    print(X_train)
    print(X_train.iloc[['Date']])
    # Train the XGBoost model
    model = train_xgboost(X_train.iloc[:][['Date']], y_train)

    # Evaluate predictions for multiple starting points
    evaluate_multiple_start_points(model, X_test, y_test, test_index, n_days_to_predict, num_start_points, feature_scaler, target_scaler)

# Run the main function
if __name__ == "__main__":
    main()



a <bound method NDFrame.keys of               Volume
Date                
2012-03-01 -1.249644
2012-03-02 -1.279547
2012-03-05 -1.295716
2012-03-06 -1.274911
2012-03-07 -1.315606
...              ...
2022-05-02 -0.402502
2022-05-03 -0.559284
2022-05-04 -0.326368
2022-05-05 -0.185128
2022-05-06 -0.439886

[2564 rows x 1 columns]>


KeyError: "None of [Index(['Date'], dtype='object')] are in the [columns]"

graph/measure of accuracy is a WIP